In [13]:
#Jesse Cooper

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
%cd /content/drive/My Drive/Colab Notebooks/gym-bandits-master

/content/drive/My Drive/Colab Notebooks/gym-bandits-master


In [16]:
pip install -e .

Obtaining file:///content/drive/My%20Drive/Colab%20Notebooks/gym-bandits-master
  Found existing installation: gym-bandits 0.0.2
    Can't uninstall 'gym-bandits'. No files were found to uninstall.
  Running setup.py develop for gym-bandits


In [17]:
import gym
import gym_bandits
import numpy as np

In [18]:
env = gym.make("BanditTwoArmedHighLowFixed-v0")

In [19]:
print(env.action_space.n)

2


In [20]:
print(env.p_dist)

[0.8, 0.2]


In [21]:
count = np.zeros(2)
sum_reward = np.zeros(2)
Q = np.zeros(2)
num_rounds =100

In [22]:
def epsilon_greedy(epsilon):
  if np.random.uniform(0,1) <epsilon:
    return env.action_space.sample()
  else:
    return np.argmax(Q)

In [23]:
for i in range(num_rounds):
  arm = epsilon_greedy(epsilon=0.5)
  next_state, reward, done, info = env.step(arm)
  count[arm] += 1
  sum_reward[arm] += reward
  Q[arm] = sum_reward[arm]/count[arm]

In [24]:
print(Q)

[0.725 0.25 ]


In [25]:
print('Optimal arm is arm {}'.format(np.argmax(Q)+1))

Optimal arm is arm 1


In [26]:
def softmax(T):
  denom = sum([np.exp(i/T) for i in Q])
  probs = [np.exp(i/T)/denom for i in Q]
  arm = np.random.choice(env.action_space.n, p=probs)
  return arm


In [27]:
T = 50

In [28]:
for i in range(num_rounds):
  arm = softmax(T)
  next_state, reward, done, info = env.step(arm)
  count[arm] += 1
  sum_reward[arm] += reward
  Q[arm] = sum_reward[arm]/count[arm]
  T = T*0.99

In [29]:
print(Q)

[0.765625 0.125   ]


In [30]:
print('Optimal arm is arm {}'.format(np.argmax(Q)+1))

Optimal arm is arm 1


Select the arm whose upper confidence bound is high

Pull the arm and receive a reward

Update the arm's mean reward and confidence interval

Repeat steps 1 to 3 for several rounds

In [31]:
def UCB(i):
  ucb= np.zeros(2)
  if i < 2:
    return i
  else:
    for arm in range(2):
      ucb[arm] = Q[arm] + np.sqrt((2*np.log(sum(count))) / count[arm])
    return (np.argmax(ucb))

In [32]:
for i in range(num_rounds):
  arm = UCB(i)
  next_state, reward, done, info = env.step(arm)
  count[arm] += 1
  sum_reward[arm] += reward
  Q[arm] = sum_reward[arm]/count[arm]

In [33]:
print(Q)

[0.77973568 0.12328767]


In [34]:
print('Optimal arm is arm {}'.format(np.argmax(Q)+1))

Optimal arm is arm 1


In [35]:
alpha = np.ones(2)
beta = np.ones(2)

In [37]:
def thompson_sampling(alpha, beta):
  samples = [np.random.beta(alpha[i] + 1,beta[i] + 1) for i in range(2)]
  return np.argmax(samples)

In [38]:
for i in range(num_rounds):
  arm = thompson_sampling(alpha, beta)
  next_state, reward, done, info = env.step(arm)
  count[arm] += 1
  sum_reward[arm] += reward
  Q[arm] = sum_reward[arm]/count[arm]
  if reward == 1:
    alpha[arm] = alpha[arm] + 1
  else:
    beta[arm] = beta[arm] + 1

In [39]:
print('Optimal arm is arm {}'.format(np.argmax(Q)+1))

Optimal arm is arm 1
